<a href="https://colab.research.google.com/github/arraiyopensource/kornia-examples/blob/master/data_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data augmentation on the GPU

In this data you learn how to use `kornia` modules in order to perform the data augmentatio on the GPU in batch mode.

## 1. Create a dummy data loader

In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [0]:
# from: https://gist.github.com/edgarriba/a781de516c508826f79568d08598efdb

class DummyDataset(Dataset):
    def __init__(self, data_root=None):
        self.data_root = data_root
        self.data_index = self.build_index(self.data_root)

    def build_index(self, data_root):
        return range(10)

    def __len__(self):
        return len(self.data_index)

    def __getitem__(self, idx):
        # get data sample
        sample = self.data_index[idx]

        # load data, NOTE: modify by cv2.imread(...)
        image = torch.rand(3, 240, 320)
        label = torch.rand(1, 240, 320)
        return dict(images=image, labels=label)

## 2. Define the data augmentation operations

Thanks to the `kornia` design all the operators can be placed inside inside a `nn.Sequential`.

In [33]:
!pip install git+https://github.com/arraiyopensource/kornia
import kornia

  Cloning https://github.com/arraiyopensource/kornia to /tmp/pip-req-build-2opte954
  Running command git clone -q https://github.com/arraiyopensource/kornia /tmp/pip-req-build-2opte954
  Created wheel for kornia: filename=kornia-0.1.2+4067336-py2.py3-none-any.whl size=110458 sha256=ff8cc4afc554f3ef21031bbacec56adc4c46ca60f8df1e8c99c484f38be7f401
  Stored in directory: /tmp/pip-ephem-wheel-cache-vdn8wu2n/wheels/a0/47/86/854e2f9a801b368f296cb25f167cbd56aa805f974b15421a67
Successfully built kornia


In [0]:
transform = nn.Sequential(
    kornia.color.AdjustBrightness(0.5),
    kornia.color.AdjustGamma(gamma=2.),
    kornia.color.AdjustContrast(0.7),
)

## 3. Run the dataset and perform the data augmentation

In [35]:
# check if cuda exists
if torch.cuda.is_available:
  device = torch.device('cuda')
else:
  device = torch.device('cpu')
print(f"Runing with device: {device}")
  
# create the dataloader
dataset = DummyDataset()
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

# get samples and perform the data augmentation
for i_batch, sample_batched in enumerate(dataloader):
    images = sample_batched['images'].to(device)
    labels = sample_batched['labels'].to(device)
    
    # perform the transforms
    images = transform(images)
    labels = transform(labels)
    
    print(f"Iteration: {i_batch} Image shape: {images.shape}")

Runing with device: cuda
Iteration: 0 Image shape: torch.Size([4, 3, 240, 320])
Iteration: 1 Image shape: torch.Size([4, 3, 240, 320])
Iteration: 2 Image shape: torch.Size([2, 3, 240, 320])
